In [ ]:
import sys
import pandas as pd
sys.path.append('..')
from khan_project_data import *

pd.set_option('display.max_rows',None)

file = "/Users/chuaixin/Desktop/pypro/static/git_match.xlsx"
df = pd.read_excel(file,sheet_name='流水线明细')

In [98]:
def self_git(ProjectID):
    """
    Purpose:获取项目内代码库 
    """
    conn = pymysql.connect(**conn_khan)
    cursor = conn.cursor()
    project_git_list = '''  
        SELECT
            PV.project_id '项目ID',
            PV.id '版本ID',
            PR.repository_id '代码库ID',
            lower(GITR.`name`) '代码库名称'
	
        FROM
            kh_project_version PV 
            LEFT JOIN kh_project__repository PR ON PV.project_id = PR.project_id
            LEFT JOIN kh_gitlab_repository GITR ON GITR.id = PR.repository_id

        WHERE
	        pv.project_id = {0} AND GITR.`name` is NOT NULL'''.format(ProjectID)
    
    cursor.execute(project_git_list)
    result = cursor.fetchall()
    # 关闭光标对象
    cursor.close()
    conn.close()
    return result
    
# end def

In [132]:
def depend_git(ProjectID):
    """
    Purpose:获取项目内代码库 
    """
    conn = pymysql.connect(**conn_khan)
    cursor = conn.cursor()
    project_git_list = '''  
        SELECT
            PV.project_id '项目ID',
            PV.id '版本ID',
            PVD.depend_project_id '依赖产品ID',
            P.name '依赖产品名称',
            PR.repository_id '代码库ID',
            lower(GITR.`name`) '代码库名称'      
        FROM
            kh_project_version PV 
            LEFT JOIN kh_project_version_dependency PVD ON PVD.project_version_id = PV.id
            LEFT JOIN kh_project__repository PR ON PVD.depend_project_id = PR.project_id
            LEFT JOIN kh_gitlab_repository GITR ON GITR.id = PR.repository_id
            LEFT JOIN kh_project p ON PVD.depend_project_id = p.id 
        WHERE
            pv.project_id = {0} AND GITR.`name` is NOT NULL
        order by PVD.depend_project_id asc; 
            '''.format(ProjectID)
    
    cursor.execute(project_git_list)
    result = cursor.fetchall()
    # 关闭光标对象
    cursor.close()
    conn.close()
    return result
    
# end def

In [233]:
grouped = df.groupby('项目ID')  
  
# 遍历每个分组并打印出来
for pid, group in grouped:
    if pid == 550:
        print(f"Value in column1: {pid}")
        print(group.index)
        print("\n---\n")  # 分隔不同的分组
        show_depend = depend_git(pid)
        print("依赖代码库数据",show_depend)  
        if show_depend:
            df_show_depend = pd.DataFrame(show_depend)  
            # 使用isin()函数检查B表的'b列'中的值是否在A表的'a列'中存在
            df.loc[group.index, '是否对应产品代码库'] = group['代码库名称'].isin(df_show_depend[5]).map({True: '匹配', False: '未匹配'})
        else:
            df.loc[group.index, '是否对应产品代码库'] = '无关联'    
    

# print(df[['项目ID','代码库名称','是否对应产品代码库']])   # 添加了这一行，以便查看结果    
df[df['项目ID']==550]

# df.to_excel('/Users/chuaixin/Desktop/pypro/static/git_match_result.xlsx')
    


Value in column1: 550
Index([372, 373, 374, 375, 376, 377, 378, 379, 380], dtype='int64')

---

依赖代码库数据 ((550, 1756, 107, '数字化研发效能平台（解决方案）', '3508', 'bit-khan-nexus'), (550, 1756, 107, '数字化研发效能平台（解决方案）', '6973', 'bit-khan-front-project'), (550, 1756, 107, '数字化研发效能平台（解决方案）', '6519', 'ultra-bi-ui'), (550, 1756, 107, '数字化研发效能平台（解决方案）', '6518', 'ultra-bi'), (550, 1756, 107, '数字化研发效能平台（解决方案）', '6515', 'ultra-msa-conf-apollo'), (550, 1756, 107, '数字化研发效能平台（解决方案）', '6514', 'ultra-uum'), (550, 1756, 107, '数字化研发效能平台（解决方案）', '6513', 'ultra-msa-pasm'), (550, 1756, 107, '数字化研发效能平台（解决方案）', '6512', 'ultra-msa-sso'), (550, 1756, 107, '数字化研发效能平台（解决方案）', '6511', 'ultra-msa-pasm-api'), (550, 1756, 107, '数字化研发效能平台（解决方案）', '6510', 'ultra-msa-gateway-api'), (550, 1756, 107, '数字化研发效能平台（解决方案）', '6509', 'ultra-msa-gateway-admin'), (550, 1756, 107, '数字化研发效能平台（解决方案）', '4450', 'khan-sdk'), (550, 1756, 107, '数字化研发效能平台（解决方案）', '4059', 'bit-khan-smartbi-arround'), (550, 1756, 107, '数字化研发效能平台（解决方案）', '3976', 'dataeas

,一级部门,二级部门,项目ID,项目名称,项目负责人,代码库ID,代码库名称,代码库地址,构建分支,是否构建,是否代码扫描,是否漏洞扫描,重复率,可靠性,安全等级,可维护性,严重漏洞数,高危漏洞数,是否对应产品代码库
372,智能运营事业部,项目五部,550,中国科学院代码三库项目,叶清,6061.0,BIT-khan-common,http://192.168.95.168:9091/nb/platform-khan/ii...,master,是,是,是,NaN,NaN,NaN,NaN,10.0,24.0,no
373,智能运营事业部,项目五部,550,中国科学院代码三库项目,叶清,6062.0,BIT-khan-test,http://192.168.95.168:9091/nb/platform-khan/ii...,master,是,是,是,NaN,NaN,NaN,NaN,0.0,0.0,no
374,智能运营事业部,项目五部,550,中国科学院代码三库项目,叶清,6063.0,BIT-khan-nexus,http://192.168.95.168:9091/nb/platform-khan/ii...,master,是,是,是,NaN,NaN,NaN,NaN,0.0,0.0,no
375,智能运营事业部,项目五部,550,中国科学院代码三库项目,叶清,6067.0,BIT-khan-dashboard,http://192.168.95.168:9091/nb/platform-khan/ii...,master,是,是,是,NaN,NaN,NaN,NaN,29.0,52.0,no
376,智能运营事业部,项目五部,550,中国科学院代码三库项目,叶清,6069.0,BIT-khan-pipeline,http://192.168.95.168:9091/nb/platform-khan/ii...,master,是,是,是,NaN,NaN,NaN,NaN,28.0,53.0,no
377,智能运营事业部,项目五部,550,中国科学院代码三库项目,叶清,6073.0,BIT-khan-project,http://192.168.95.168:9091/nb/platform-khan/ii...,master,是,是,是,NaN,NaN,NaN,NaN,30.0,53.0,no
378,智能运营事业部,项目五部,550,中国科学院代码三库项目,叶清,6075.0,BIT-khan-document,http://192.168.95.168:9091/nb/platform-khan/ii...,master,是,是,是,NaN,NaN,NaN,NaN,0.0,0.0,no
379,智能运营事业部,项目五部,550,中国科学院代码三库项目,叶清,6076.0,BIT-khan-gitlab,http://192.168.95.168:9091/nb/platform-khan/ii...,master,是,是,是,NaN,NaN,NaN,NaN,29.0,54.0,no
380,智能运营事业部,项目五部,550,中国科学院代码三库项目,叶清,6080.0,BIT-khan-front,http://192.168.95.168:9091/nb/platform-khan/ii...,master,是,是,是,NaN,NaN,NaN,NaN,7.0,12.0,no


In [229]:
pid = 550
show = self_git(pid)
show_depend = depend_git(pid)

# print(*show,sep='\n') 
# print(*show_depend,sep='\n') 

#通过pandas合并show和show_depend，对比最后一列，标识出匹配情况
# 将 show 和 show_depend 转换为 pandas DataFrame

if show:
    df_show = pd.DataFrame(show)
    if show_depend:
        df_show_depend = pd.DataFrame(show_depend)

        # 使用isin()函数检查B表的'b列'中的值是否在A表的'a列'中存在
        df_show_depend['exists'] = df_show_depend[5].isin(df_show[3])
        # 将True/False转换为'exist'/'no'
        df_show_depend['exists'] = df_show_depend['exists'].map({True: 'exist', False: 'no'})
        print("产品依赖匹配\n",df_show_depend)

        # 使用isin()函数检查B表的'b列'中的值是否在A表的'a列'中存在
        df_show['exists'] = df_show[3].isin(df_show_depend[5])
        # 将True/False转换为'exist'/'no'
        df_show['exists'] = df_show['exists'].map({True: 'exist', False: 'no'})
        print("本项目代码库匹配\n",df_show)
    else:
        print("无产品依赖配置")     
else:
    print('无代码库')
# end if


产品依赖匹配
       0     1    2                3     4                               5  \
0   550  1756  107  数字化研发效能平台（解决方案）  3508                  bit-khan-nexus   
1   550  1756  107  数字化研发效能平台（解决方案）  6973          bit-khan-front-project   
2   550  1756  107  数字化研发效能平台（解决方案）  6519                     ultra-bi-ui   
3   550  1756  107  数字化研发效能平台（解决方案）  6518                        ultra-bi   
4   550  1756  107  数字化研发效能平台（解决方案）  6515           ultra-msa-conf-apollo   
5   550  1756  107  数字化研发效能平台（解决方案）  6514                       ultra-uum   
6   550  1756  107  数字化研发效能平台（解决方案）  6513                  ultra-msa-pasm   
7   550  1756  107  数字化研发效能平台（解决方案）  6512                   ultra-msa-sso   
8   550  1756  107  数字化研发效能平台（解决方案）  6511              ultra-msa-pasm-api   
9   550  1756  107  数字化研发效能平台（解决方案）  6510           ultra-msa-gateway-api   
10  550  1756  107  数字化研发效能平台（解决方案）  6509         ultra-msa-gateway-admin   
11  550  1756  107  数字化研发效能平台（解决方案）  4450                        kha

<!-- show = depend_git(pid)  

print(*show,sep='\n')  -->